In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
from matplotlib.dates import DateFormatter, WeekdayLocator,\
    DayLocator, MONDAY
from matplotlib.finance import quotes_historical_yahoo_ohlc, candlestick_ohlc
import time
import sys
import numpy as np
from statsmodels.tsa.arima_model import ARIMA
from pandas.tseries.offsets import *
import statsmodels.api as sm

In [2]:
def getdata():
#get data from 5000days ago which got 3569 entries
    date1 = time.strftime("%Y-%m-%d").split('-');
    date2 = datetime.datetime.now() - datetime.timedelta(days=5000)
    date2 = date2.strftime("%Y-%m-%d").split('-');
    date1 = [int(i) for i in date1]
    date2 = [int(i) for i in date2]
    date1 = (date1[0],date1[1],date1[2]);
    date2 = (date2[0],date2[1],date2[2]);
    start = date2;
    end = date1;
    mondays = WeekdayLocator(MONDAY)        # major ticks on the mondays
    alldays = DayLocator()              # minor ticks on the days
    weekFormatter = DateFormatter('%b %d')  # e.g., Jan 12
    dayFormatter = DateFormatter('%d')      # e.g., 12
    try:
        quotes = quotes_historical_yahoo_ohlc('CNY=X', start, end)
        if len(quotes) == 0:
            print(0);
            raise SystemExit
        data = list(quotes);
        for i in range(0,len(data)):
            data[i] = list(data[i]);
            data[i][0] = datetime.datetime.fromordinal(int(data[i][0])).strftime('%Y-%m-%d');
        return data
    except:
        print(0);
        return 0;

In [3]:
def predict (steps):
    #dateparse = lambda dates: pd.datetime.strptime(dates, '%m/%d/%y')
    #data = pd.read_csv('rateData.csv', parse_dates=True, index_col='Date',date_parser=dateparse)
    #ts = data['RMBperUSD']
    #new_period = ts['2015-08-11':]
    
    data = getdata();
    data = [[pd.datetime.strptime(x[0],'%Y-%m-%d')   ,x[4]]\
        for x in data]
    col = ['date','RMBperUSD']
    data = pd.DataFrame.from_records(data,columns=col,index = col[0]);
    new_period = data['RMBperUSD']
    
    new_period_log = np.log(new_period)
       
    model = ARIMA(new_period_log, order=(2, 1, 2),freq='D')
    results_ARIMA = model.fit(disp=-1)

    predictions_ARIMA_diff = pd.Series(results_ARIMA.fittedvalues, copy=True)
    startDate = new_period.index[-1]
    endDate = new_period.index[-1] + Day(steps)
    
    ##dynamic_Sample_Prediction is error is future
    dynamic_Sample_Prediction = results_ARIMA.predict(startDate.isoformat(), endDate.isoformat(), exog = None, dynamic = False)

    ## dynamic_Sample_Prediction_log is error data with future error
    dynamic_Sample_Prediction_log = dynamic_Sample_Prediction.add(predictions_ARIMA_diff,fill_value=0)

    ## dynamic_Sample_Prediction_log_cumsum is cumulate sum of all errors
    dynamic_Sample_Prediction_log_cumsum = dynamic_Sample_Prediction_log.cumsum()

    ## expand 1.81094 to date + future date
    dynamic_ARIMA_log = pd.Series(new_period_log.ix[0], index=dynamic_Sample_Prediction_log.index)

    ## add 1.81094 to all cumsum respectively
    dynamic_ARIMA_log = dynamic_ARIMA_log.add(dynamic_Sample_Prediction_log_cumsum,fill_value=0)

    ## back to original data
    dynamic_Sample_Prediction_ARIMA = np.exp(dynamic_ARIMA_log)

    ##prediction data
    predictResult = dynamic_Sample_Prediction_ARIMA[new_period.index[-1]:]
    datelist = predictResult.index.tolist();
    datelist = [time.strftime("%Y-%m-%d",time.strptime(str(i),"%Y-%m-%d %H:%M:%S"))\
            for i in datelist]
    ratelist = predictResult.values.tolist();
    ans = []
    for i in range(0,len(datelist)):
        ans.append([datelist[i],ratelist[i]])
    return ans

In [4]:
ans = 0;
try:
    days = int(sys.argv[1])
    ans = predict(days)
except:
    ans = 0;
print(ans);

0


In [5]:
import math
def accu (steps):

	
	data = getdata();
	data = [[pd.datetime.strptime(x[0],'%Y-%m-%d')   ,x[4]]\
		for x in data]
	col = ['date','RMBperUSD']
	data = pd.DataFrame.from_records(data,columns=col,index = col[0]);
	new_period = data['RMBperUSD']
	new_period_log = np.log(new_period)
	
	

	#define train data
	new_period_train = new_period[:int(len(new_period_log)*0.8)]
	train = new_period_log[0:int(len(new_period_log)*0.8)]

	train_model = ARIMA(train, order=(2, 1, 2),freq='D')
	train_results_ARIMA = train_model.fit(disp=-1)

	train_predictions_ARIMA_diff = pd.Series(train_results_ARIMA.fittedvalues, copy=True)
	startDate = new_period_train.index[-1]
	endDate = new_period_train.index[-1] + Day(steps)

	train_dynamic_Sample_Prediction = train_results_ARIMA.predict(startDate.isoformat(), endDate.isoformat(), exog = None, dynamic = False)
	train_dynamic_Sample_Prediction_log = train_dynamic_Sample_Prediction.add(train_predictions_ARIMA_diff,fill_value=0)
	train_dynamic_Sample_Prediction_log_cumsum = train_dynamic_Sample_Prediction_log.cumsum()
	train_dynamic_ARIMA_log = pd.Series(train.ix[0], index=train_dynamic_Sample_Prediction_log.index)
	train_dynamic_ARIMA_log = train_dynamic_ARIMA_log.add(train_dynamic_Sample_Prediction_log_cumsum,fill_value=0)
	train_dynamic_Sample_Prediction_ARIMA = np.exp(train_dynamic_ARIMA_log)

	train_predictResult = train_dynamic_Sample_Prediction_ARIMA[new_period_train.index[-1]:]
	train_datelist = train_predictResult.index.tolist();

	train_datelist = [time.strftime("%Y-%m-%d",time.strptime(str(i),"%Y-%m-%d %H:%M:%S"))\
			for i in train_datelist]
 
	train_ratelist = train_predictResult.values.tolist()

	train_ans = []
	for i in range(0,len(train_datelist)):
		train_ans.append([train_datelist[i],train_ratelist[i]])

	#define test data
	test_data = data[int(len(data)*0.8):]

	#calculate rmse
	diff = []
	for i in range(1,steps+1):
		diff.append((train_ans[i][1] - test_data.iloc[i-1]) ** 2)
	
	rmse = math.sqrt(np.mean(diff))

	return rmse

In [6]:
accu(5)

/Users/lingxiangyu/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/lingxiangyu/anaconda/lib/python2.7/site-packages/statsmodels/base/data.py:503: FutureWarning: TimeSeries is deprecated. Please use Series
  return TimeSeries(result, index=self.predict_dates)


0.012641300030221226

In [7]:
predict(5)

[['2017-04-26', 6.8646717155272885],
 ['2017-04-27', 6.847813818659736],
 ['2017-04-28', 6.850912662715539],
 ['2017-04-29', 6.850260027553502],
 ['2017-04-30', 6.8497946040967665],
 ['2017-05-01', 6.849497123904665]]

In [8]:
accu(10)

/Users/lingxiangyu/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0.020044455604420485

In [9]:
predict(10)

[['2017-04-26', 6.8646717155272885],
 ['2017-04-27', 6.847813818659736],
 ['2017-04-28', 6.850912662715539],
 ['2017-04-29', 6.850260027553502],
 ['2017-04-30', 6.8497946040967665],
 ['2017-05-01', 6.849497123904665],
 ['2017-05-02', 6.849121863861703],
 ['2017-05-03', 6.848766294928698],
 ['2017-05-04', 6.848408042196151],
 ['2017-05-05', 6.848049601849873],
 ['2017-05-06', 6.847691434809505]]

In [10]:
accu(50)

/Users/lingxiangyu/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0.027066827648813848

In [11]:
predict(50)

[['2017-04-26', 6.8646717155272885],
 ['2017-04-27', 6.847813818659736],
 ['2017-04-28', 6.850912662715539],
 ['2017-04-29', 6.850260027553502],
 ['2017-04-30', 6.8497946040967665],
 ['2017-05-01', 6.849497123904665],
 ['2017-05-02', 6.849121863861703],
 ['2017-05-03', 6.848766294928698],
 ['2017-05-04', 6.848408042196151],
 ['2017-05-05', 6.848049601849873],
 ['2017-05-06', 6.847691434809505],
 ['2017-05-07', 6.847333200080807],
 ['2017-05-08', 6.846975001573277],
 ['2017-05-09', 6.846616820514177],
 ['2017-05-10', 6.846258657573615],
 ['2017-05-11', 6.845900513694649],
 ['2017-05-12', 6.845542388463438],
 ['2017-05-13', 6.845184281979998],
 ['2017-05-14', 6.84482619423034],
 ['2017-05-15', 6.844468125212085],
 ['2017-05-16', 6.844110074925638],
 ['2017-05-17', 6.843752043369566],
 ['2017-05-18', 6.843394030542977],
 ['2017-05-19', 6.843036036444886],
 ['2017-05-20', 6.842678061074312],
 ['2017-05-21', 6.842320104430273],
 ['2017-05-22', 6.841962166511793],
 ['2017-05-23', 6.841604247

In [12]:
accu(100)

/Users/lingxiangyu/anaconda/lib/python2.7/site-packages/statsmodels/base/model.py:466: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


0.029025027074880014

In [13]:
predict(100)

[['2017-04-26', 6.8646717155272885],
 ['2017-04-27', 6.847813818659736],
 ['2017-04-28', 6.850912662715539],
 ['2017-04-29', 6.850260027553502],
 ['2017-04-30', 6.8497946040967665],
 ['2017-05-01', 6.849497123904665],
 ['2017-05-02', 6.849121863861703],
 ['2017-05-03', 6.848766294928698],
 ['2017-05-04', 6.848408042196151],
 ['2017-05-05', 6.848049601849873],
 ['2017-05-06', 6.847691434809505],
 ['2017-05-07', 6.847333200080807],
 ['2017-05-08', 6.846975001573277],
 ['2017-05-09', 6.846616820514177],
 ['2017-05-10', 6.846258657573615],
 ['2017-05-11', 6.845900513694649],
 ['2017-05-12', 6.845542388463438],
 ['2017-05-13', 6.845184281979998],
 ['2017-05-14', 6.84482619423034],
 ['2017-05-15', 6.844468125212085],
 ['2017-05-16', 6.844110074925638],
 ['2017-05-17', 6.843752043369566],
 ['2017-05-18', 6.843394030542977],
 ['2017-05-19', 6.843036036444886],
 ['2017-05-20', 6.842678061074312],
 ['2017-05-21', 6.842320104430273],
 ['2017-05-22', 6.841962166511793],
 ['2017-05-23', 6.841604247